##  `Steane code - a Quantum Error-correcting code`  

In this notebook we will construct a function that takes two imputs: $x\in \mathbb{F}^n_2$ and an error probability $p\in(0,1)$ and outputs the logical state $\ket{x_L}$ in Steane's code. 
We will then run it thuoght a Pauli Error Channel, with error rate $p$, measures syndromes, applies the recovery operation when needed, measure the data qubits and decodes the results.

Recall that Steane's code is a 7 qubits code to encode 1 logical qubit, it can correct any single-qubit error $(X,Y,Z)$. It has distance 3 which means that it can detect up to two erros and correct one. It is insipired from the CCS $[7,4,3]$ Hamming code. It is constructed from two classical linear coodes, one for correcting bit-flip errors and one for correcting phase-flip errors.

We will proceed as follows, first the Encoding steps:
   - The logical qubit is encoded into **7 physical qubits** using combinations of codewords from the classical **[7,4,3] Hamming code**.
   - This encoding spreads the information across multiple qubits, enabling protection against single-qubit errors.

Next the Error correction:

- **Syndrome measurement:** Separate syndromes are measured to detect **bit-flip (X)** and **phase-flip (Z)** errors.  
- These measurements are performed using stabilizer operators derived from the classical code's **parity check matrix**.
- **Error identification and correction:** The measured syndrome indicates the location and type of error, allowing it to be corrected by applying the appropriate **Pauli operator** (X, Y, or Z).

For the construction we will follow https://errorcorrectionzoo.org/c/steane

In [10]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, AncillaRegister
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.visualization import plot_histogram

H = np.array([
    [0, 0, 0, 1, 1, 1, 1],
    [0, 1, 1, 0, 0, 1, 1],
    [1, 0, 1, 0, 1, 0, 1]])

print("H matrix:")
print(H)

H matrix:
[[0 0 0 1 1 1 1]
 [0 1 1 0 0 1 1]
 [1 0 1 0 1 0 1]]


The above matrix is teh parity check matric of the [7,4,3] Hamming code, the stabilizer group of Steane code has six generator, three of X-type and three of Z-type. First we will prepare the $0$ logical state using hadamard gate as well as CNOT gates matched with H. 

The next encoder is also taken from [the website](https://errorcorrectionzoo.org/c/steane)

In [22]:
sevenqubits = QuantumRegister(size=7, name="7_qubits")
encoding = QuantumCircuit(sevenqubits)

encoding.h(0)
encoding.cx(sevenqubits[6], sevenqubits[4])
encoding.cx(sevenqubits[6], sevenqubits[5])
encoding.cx(sevenqubits[0], sevenqubits[3])
encoding.cx(sevenqubits[0], sevenqubits[5])
encoding.cx(sevenqubits[0], sevenqubits[6])
encoding.barrier()
encoding.h(1)
encoding.cx(sevenqubits[1], sevenqubits[3])
encoding.cx(sevenqubits[1], sevenqubits[4])
encoding.cx(sevenqubits[1], sevenqubits[6])
encoding.barrier()
encoding.h(2)
encoding.cx(sevenqubits[2], sevenqubits[3])
encoding.cx(sevenqubits[2], sevenqubits[4])
encoding.cx(sevenqubits[2], sevenqubits[5])
encoding.barrier()

decoding = encoding.inverse()

encoding.draw()

┌───┐                ░                      ░                     »
7_qubits_0: ┤ H ├──■────■────■───░──────────────────────░─────────────────────»
            └───┘  │    │    │   ░ ┌───┐                ░                     »
7_qubits_1: ───────┼────┼────┼───░─┤ H ├──■────■────■───░─────────────────────»
                   │    │    │   ░ └───┘  │    │    │   ░ ┌───┐               »
7_qubits_2: ───────┼────┼────┼───░────────┼────┼────┼───░─┤ H ├──■────■────■──»
                 ┌─┴─┐  │    │   ░      ┌─┴─┐  │    │   ░ └───┘┌─┴─┐  │    │  »
7_qubits_3: ─────┤ X ├──┼────┼───░──────┤ X ├──┼────┼───░──────┤ X ├──┼────┼──»
            ┌───┐└───┘  │    │   ░      └───┘┌─┴─┐  │   ░      └───┘┌─┴─┐  │  »
7_qubits_4: ┤ X ├───────┼────┼───░───────────┤ X ├──┼───░───────────┤ X ├──┼──»
            └─┬─┘┌───┐┌─┴─┐  │   ░           └───┘  │   ░           └───┘┌─┴─┐»
7_qubits_5: ──┼──┤ X ├┤ X ├──┼───░──────────────────┼───░────────────────┤ X ├»
              │  └─┬─┘└───┘┌─┴─┐ ░                ┌─┴─┐ ░                └───┘»
7_qubits_6: ──■────■───────┤ X ├─░────────────────┤ X ├─░─────────────────────»
                           └───┘ ░                └───┘ ░                     »
«             ░ 
«7_qubits_0: ─░─
«             ░ 
«7_qubits_1: ─░─
«             ░ 
«7_qubits_2: ─░─
«             ░ 
«7_qubits_3: ─░─
«             ░ 
«7_qubits_4: ─░─
«             ░ 
«7_qubits_5: ─░─
«             ░ 
«7_qubits_6: ─░─
«             ░